In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import tensorflow as tf
import timeit
import os
from six.moves import cPickle
from text_utils import TextLoader
from tensorflow.contrib import rnn
from char_rnn_model import Model

In [3]:
import tensorflow as tf
from tensorflow.contrib import layers
from tensorflow.contrib import rnn  # rnn stuff temporarily in contrib, moving back to code in TF 1.1
import os
import time
import math
import numpy as np
import my_txtutils as txt
tf.set_random_seed(0)

# Define directories, hyperparameter

In [4]:
SEQLEN = 30
BATCHSIZE = 200
ALPHASIZE = txt.ALPHASIZE
INTERNALSIZE = 512
NLAYERS = 3
learning_rate = 0.001  # fixed learning rate
dropout_pkeep = 0.8    # some dropout


# Load data using TextLoader object

In [5]:
shakedir = "shakespeare.txt"
#shakedir = "../tensorflow/**/*.py"
codetext, valitext, bookranges = txt.read_data_files(shakedir, validation=True)

# display some stats on the data
epoch_size = len(codetext) // (BATCHSIZE * SEQLEN)
txt.print_data_stats(len(codetext), len(valitext), epoch_size)

Loading file shakespeare.txt
Training text size is 4.36MB with 0.00KB set aside for validation. There will be 762 batches per epoch


# Create your model object

In [6]:
lr = tf.placeholder(tf.float32, name='lr') 
pkeep = tf.placeholder(tf.float32, name='pkeep') 
batchsize = tf.placeholder(tf.int32, name='batchsize')

In [7]:
# inputs
X = tf.placeholder(tf.uint8, [None, None], name='X')    # [ BATCHSIZE, SEQLEN ]
Xo = tf.one_hot(X, ALPHASIZE, 1.0, 0.0)                 # [ BATCHSIZE, SEQLEN, ALPHASIZE ]
# expected outputs = same sequence shifted by 1 since we are trying to predict the next character
Y_ = tf.placeholder(tf.uint8, [None, None], name='Y_')  # [ BATCHSIZE, SEQLEN ]
Yo_ = tf.one_hot(Y_, ALPHASIZE, 1.0, 0.0)               # [ BATCHSIZE, SEQLEN, ALPHASIZE ]
# input state
Hin = tf.placeholder(tf.float32, [None, INTERNALSIZE*NLAYERS], name='Hin')  # [ BATCHSIZE, INTERNALSIZE * NLAYERS]

In [8]:
cells = [rnn.GRUCell(INTERNALSIZE) for _ in range(NLAYERS)]

dropcells = [rnn.DropoutWrapper(cell,input_keep_prob=pkeep) for cell in cells]
multicell = rnn.MultiRNNCell(dropcells, state_is_tuple=False)
multicell = rnn.DropoutWrapper(multicell, output_keep_prob=pkeep)

Yr, H = tf.nn.dynamic_rnn(multicell, Xo, dtype=tf.float32, initial_state=Hin)

H = tf.identity(H, name='H') 

In [15]:
Yflat = tf.reshape(Yr, [-1, INTERNALSIZE])    # [ BATCHSIZE x SEQLEN, INTERNALSIZE ]
Ylogits = layers.linear(Yflat, ALPHASIZE)     # [ BATCHSIZE x SEQLEN, ALPHASIZE ]
Yflat_ = tf.reshape(Yo_, [-1, ALPHASIZE])     # [ BATCHSIZE x SEQLEN, ALPHASIZE ]
loss = tf.nn.softmax_cross_entropy_with_logits_v2(logits=Ylogits, labels=Yflat_)  # [ BATCHSIZE x SEQLEN ]
loss = tf.reshape(loss, [batchsize, -1])      # [ BATCHSIZE, SEQLEN ]
Yo = tf.nn.softmax(Ylogits, name='Yo')        # [ BATCHSIZE x SEQLEN, ALPHASIZE ]
Y = tf.argmax(Yo, 1)                          # [ BATCHSIZE x SEQLEN ]
Y = tf.reshape(Y, [batchsize, -1], name="Y")  # [ BATCHSIZE x SEQLEN ]
train_step = tf.train.AdamOptimizer(lr).minimize(loss)

In [16]:
# stats for display
seqloss = tf.reduce_mean(loss, 1)
batchloss = tf.reduce_mean(seqloss)
accuracy = tf.reduce_mean(tf.cast(tf.equal(Y_, tf.cast(Y, tf.uint8)), tf.float32))
loss_summary = tf.summary.scalar("batch_loss", batchloss)
acc_summary = tf.summary.scalar("batch_accuracy", accuracy)
summaries = tf.summary.merge([loss_summary, acc_summary])

In [ ]:
timestamp = str(math.trunc(time.time()))
summary_writer = tf.summary.FileWriter("log/" + timestamp + "-training")
validation_writer = tf.summary.FileWriter("log/" + timestamp + "-validation")

In [ ]:
if not os.path.exists("checkpoints"):
    os.mkdir("checkpoints")
saver = tf.train.Saver(max_to_keep=1000)

In [17]:
DISPLAY_FREQ = 50
_50_BATCHES = DISPLAY_FREQ * BATCHSIZE * SEQLEN
progress = txt.Progress(DISPLAY_FREQ, size=111+2, msg="Training on next "+str(DISPLAY_FREQ)+" batches")

In [18]:
from tensorflow_graph_in_jupyter import show_graph
show_graph(tf.get_default_graph())

# Training

In [19]:
# init
istate = np.zeros([BATCHSIZE, INTERNALSIZE*NLAYERS])  # initial zero input state
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
step = 0

In [21]:
# training loop
for x, y_, epoch in txt.rnn_minibatch_sequencer(codetext, BATCHSIZE, SEQLEN, nb_epochs=10):

    # train on one minibatch
    feed_dict = {X: x, Y_: y_, Hin: istate, lr: learning_rate, pkeep: dropout_pkeep, batchsize: BATCHSIZE}
    _, y, ostate = sess.run([train_step, Y, H], feed_dict=feed_dict)

    # log training data for Tensorboard display a mini-batch of sequences (every 50 batches)
    if step % _50_BATCHES == 0:
        feed_dict = {X: x, Y_: y_, Hin: istate, pkeep: 1.0, batchsize: BATCHSIZE}  # no dropout for validation
        y, l, bl, acc, smm = sess.run([Y, seqloss, batchloss, accuracy, summaries], feed_dict=feed_dict)
        txt.print_learning_learned_comparison(x, y, l, bookranges, bl, acc, epoch_size, step, epoch)
        summary_writer.add_summary(smm, step)

    # run a validation step every 50 batches
    # The validation text should be a single sequence but that's too slow (1s per 1024 chars!),
    # so we cut it up and batch the pieces (slightly inaccurate)
    # tested: validating with 5K sequences instead of 1K is only slightly more accurate, but a lot slower.
    if step % _50_BATCHES == 0 and len(valitext) > 0:
        VALI_SEQLEN = 1*1024  # Sequence length for validation. State will be wrong at the start of each sequence.
        bsize = len(valitext) // VALI_SEQLEN
        txt.print_validation_header(len(codetext), bookranges)
        vali_x, vali_y, _ = next(txt.rnn_minibatch_sequencer(valitext, bsize, VALI_SEQLEN, 1))  # all data in 1 batch
        vali_nullstate = np.zeros([bsize, INTERNALSIZE*NLAYERS])
        feed_dict = {X: vali_x, Y_: vali_y, Hin: vali_nullstate, pkeep: 1.0,  # no dropout for validation
                     batchsize: bsize}
        ls, acc, smm = sess.run([batchloss, accuracy, summaries], feed_dict=feed_dict)
        txt.print_validation_stats(ls, acc)
        # save validation data for Tensorboard
        validation_writer.add_summary(smm, step)

    # display a short text generated with the current weights and biases (every 150 batches)
    if step // 3 % _50_BATCHES == 0:
        txt.print_text_generation_header()
        ry = np.array([[txt.convert_from_alphabet(ord("K"))]])
        rh = np.zeros([1, INTERNALSIZE * NLAYERS])
        for k in range(1000):
            ryo, rh = sess.run([Yo, H], feed_dict={X: ry, pkeep: 1.0, Hin: rh, batchsize: 1})
            rc = txt.sample_from_probabilities(ryo, topn=10 if epoch <= 1 else 2)
            print(chr(txt.convert_to_alphabet(rc)), end="")
            ry = np.array([[rc]])
        txt.print_text_generation_footer()

    # save a checkpoint (every 500 batches)
    if step // 10 % _50_BATCHES == 0:
        saved_file = saver.save(sess, 'checkpoints/rnn_train_' + timestamp, global_step=step)
        print("Saved file: " + saved_file)

    # display progress bar
    progress.step(reset=step % _50_BATCHES == 0)

    # loop state around
    istate = ostate
    step += BATCHSIZE * SEQLEN

600000 (epoch 0) shakespeare.txt │ t they think we are too dear:  │  eee  hee   he ee  ee  e     h │ loss: 2.49905
600030 (epoch 0) shakespeare.txt │ \LARTIUS:\Worthy sir, thou ble │ AAA:::::: he    eee  tee  eee  │ loss: 2.70192
600060 (epoch 0) shakespeare.txt │ ome time when his soaring inso │    eee  eee  ee  ee      ee    │ loss: 2.64608
600090 (epoch 0) shakespeare.txt │ I be consul? By yond clouds,\L │  ho eee    \te eee  eee        │ loss: 3.11278
600120 (epoch 0) shakespeare.txt │ rs, which show\Like graves i'  │    hoe   eee       hee e  ee e │ loss: 3.12852
600150 (epoch 0) shakespeare.txt │ ond Servingman:\Why, then we s │    he  e        ho  hhe  ee ee │ loss: 3.09300
600180 (epoch 0) shakespeare.txt │ ing those that eye thee!\\VOLU │ e  eee   hee  ee  hee  \\AAAA: │ loss: 2.69856
600210 (epoch 0) shakespeare.txt │ :\'Tis very grievous to be tho │   h   he   eeee     he ee eee  │ loss: 3.01069
600240 (epoch 0) shakespeare.txt │ h;\And, after many lengthen'd  │   \    hh   

900000 (epoch 0) shakespeare.txt │ What work's, my countrymen, in │ het hou     ao tout  e e   an  │ loss: 2.30714
900030 (epoch 0) shakespeare.txt │ ing thy news so late?\\Messeng │ t  the ao   aoutot  \\IO       │ loss: 2.48304
900060 (epoch 0) shakespeare.txt │ st Officer:\If he did not care │   ho  rh   I  he tot tou hhre  │ loss: 2.57451
900090 (epoch 0) shakespeare.txt │ ple,\As if you were a god to p │  e  Tn tn tou th e tntou thuto │ loss: 2.27006
900120 (epoch 0) shakespeare.txt │ he extremest death.\\BRUTUS:\B │ e trehe e   ao t e\\IOENEN:\Io │ loss: 2.48902
900150 (epoch 0) shakespeare.txt │  would do,\Were he more angry  │ thut  tou Th   te teu  tnd   a │ loss: 2.15551
900180 (epoch 0) shakespeare.txt │ hereto we are bound, together  │ e     to tn  tou    ahu   e  a │ loss: 2.20139
900210 (epoch 0) shakespeare.txt │ d that made these fatal holes! │  thet het  the   aorhreteu   \ │ loss: 2.07253
900240 (epoch 0) shakespeare.txt │ by me, and ends in 'Margaret.' │ o to  and to

SWoyolise thest tiur tomer with thuse af theuthout.
IONNIO::Nhich bo ou toun whit at th umee sarlas the mit thhe an tor, his mer benar

Aer Iimlen mo hads, toot teu he seotier;
Whas thet se and bh eu the anls fure the ce the sytthe thas art eis tast her anst besichesasdian morl,
hos haver, of hete be thin hemand tes sate his worlos

Sice mind ariste bh ine, hhaud, whys he shiardsins

Ard bhat an sa sotre fooc mh shars oucerorir.

IANEA:
I el toeus sare ouund thy als bir, hat in tho batt hrunsir mr ind
Ter in then brasse my aus me bholesteer.:
Mndd I w thot me ande atstan aredtener ofo athithe.

PAINES
To thou do tomd thu ll hir shat hhsrowutisl hhe miuge.

Mond iro aod oort,
The mans so haud batt hames, anout yout hier heu h ellsetin sothile werhe sishe
annd woure, worsinton doud wath hamise af mit hutt the the hon he hind bus hims hhar thath hore.

PESINT:
Wher the, the hamd bu me whar bfith an hal m us anthon of thathor
Iath at boud birshane to usathe
I ariee bivs bare. Bra
└────────

1202340 (epoch 0) shakespeare.txt │ the wandering wind.\Blows dust │  e wird   ng tong \\oo e tor   │ loss: 2.09837
1202370 (epoch 0) shakespeare.txt │  loves you well that holds his │ houe  aou horl when heul  aas  │ loss: 1.76209
1202400 (epoch 0) shakespeare.txt │  viol once more: how thou stir │ sesn tf e mene  Ier hheu woone │ loss: 2.26868
1202430 (epoch 0) shakespeare.txt │ are bound to him\indeed; but h │ nd te rd th mas \n  r   aet te │ loss: 2.05644
1202460 (epoch 0) shakespeare.txt │ \Our son and daughter shall in │ \n  to dtnd tonn     toenl wn  │ loss: 2.16030
1202490 (epoch 0) shakespeare.txt │ chains\And faster bound to Aar │ oann  \nd tor  r te r  th mnd  │ loss: 2.13257
1202520 (epoch 0) shakespeare.txt │  lament in vain:\The tribunes  │ wove   tn tenn \Tha shese    t │ loss: 2.29269
1202550 (epoch 0) shakespeare.txt │  a verse in Horace; I know it  │ tnte e  tn tou ne \a wo w tn h │ loss: 2.28116
1202580 (epoch 0) shakespeare.txt │  told the empress of this spor │ sh 

1500000 (epoch 0) shakespeare.txt │ w speaks! What then? what then │  thaar  \ahat the   Ihet the   │ loss: 2.01711
1500030 (epoch 0) shakespeare.txt │ ome other fight,\As cause will │  e tf er torhe \An tonnt titl  │ loss: 1.89766
1500060 (epoch 0) shakespeare.txt │ pawn how can he flatter--\That │ rr  tir hondterhoorher \\Ahet  │ loss: 2.51312
1500090 (epoch 0) shakespeare.txt │ t without all reason, where ge │  tith u  tnl tear    aiir  toa │ loss: 2.00643
1500120 (epoch 0) shakespeare.txt │  bear;\That when the sea was c │ te r \Ahet sii  the soartil to │ loss: 1.83775
1500150 (epoch 0) shakespeare.txt │ \Upon our territories; and hav │ \nar tfr thaeenh en   and tive │ loss: 2.24769
1500180 (epoch 0) shakespeare.txt │ Nay, behold 's:\This boy, that │ ot  te eue ts  Thet ter  thet  │ loss: 2.23438
1500210 (epoch 0) shakespeare.txt │ ese known evils, but to give m │    tedw  tnenl  aet th tone te │ loss: 2.21876
1500240 (epoch 0) shakespeare.txt │ garet was a prophetess!\Live e │ ene

...

Master with his tongue, thou dost?
To arm somedgors he so much in the somest, she's frauth,
Are till a semstation this brom hone.

PRINCESS:
And then you, she honest is to me?

FELDIA:
You do your dien?

KING HINRY:
Where'f your watchis from marces? mark a more thou constites:
Well, see, well, say these bath madied be
As buck to dusting.

CHSSPER:
It was still a found: but I shall heave they wrong thy for it:
Thou daughter if not for his father that suthent heart wither, we woll wive
a liet. Be not them

I would wear we like of a life, and all thou she that he dow
She had made it to the facter.

BULON:
Homest belle de mosely; but you would not morr,
Who hath me willow they swoll me cantling at my tongue?

SAYA:
Well; my glory waste, much both remuil.

MARIA:
Ay! she thin thy bosiness made wither;
Ship woman's aring, bear your folty answer.'

AARON:
He die ale fault be was the misher.

DEMIORENUS:
To speak you, sorstanding in, my lord.

DUKE OP YONK:
Ambeal, I leen you slay.

└─────────

6900000 (epoch 1) shakespeare.txt │ and from this time,\For what h │ nd toom thes shme,\Aor that te │ loss: 0.97744
6900030 (epoch 1) shakespeare.txt │ le deeds, we must also tell hi │ e tead   ah wast n lenthll tim │ loss: 1.46480
6900060 (epoch 1) shakespeare.txt │ m it be put to that?\\First Se │ etn ie sar oo thet \\CArst Men │ loss: 1.31877
6900090 (epoch 1) shakespeare.txt │ UTUS:\Pray, let us go.\\VOLUMN │ TUS:\Iray  yet ms too\\CILENII │ loss: 1.33736
6900120 (epoch 1) shakespeare.txt │  sir,\Even by your own.\\AUFID │ air, Tnen te tour snn \\CNCELI │ loss: 1.57744
6900150 (epoch 1) shakespeare.txt │ full. You have pray'd well to- │ orl  Iou save saoy d ti l ah m │ loss: 1.51665
6900180 (epoch 1) shakespeare.txt │ ike father, like a child,\Told │ ne oother  weke a soald \Th d  │ loss: 1.24308
6900210 (epoch 1) shakespeare.txt │ To yield the ghost: but still  │ h soeld the wrele  wut thall t │ loss: 1.46077
6900240 (epoch 1) shakespeare.txt │ preserved, cherish'd, and kept │ ees

7200000 (epoch 1) shakespeare.txt │ US:\Condition!\I would I were  │ S:\Tomsenion, \ wiuld n hire n │ loss: 1.19737
7200030 (epoch 1) shakespeare.txt │ own of humility: mark his\beha │ rn of tisbneny.\Iykr yim seail │ loss: 2.04105
7200060 (epoch 1) shakespeare.txt │ peak 'em fair?\\SICINIUS:\Wher │ eak ttm torr \\PIRINIUS:\Thane │ loss: 1.38862
7200090 (epoch 1) shakespeare.txt │ en? Our state thinks not\so: t │   \I r soanesohesks aot to  th │ loss: 1.84938
7200120 (epoch 1) shakespeare.txt │ efect of judgment,\To fail in  │  ont tf tusgeent \Th sair tn t │ loss: 1.40895
7200150 (epoch 1) shakespeare.txt │ 's difference; but the fall of │ s aeseerence  aut Ihe sail of  │ loss: 1.17010
7200180 (epoch 1) shakespeare.txt │ uld I knew thy heart.\\GLOUCES │ ld h snow the feart \\KLOUCEST │ loss: 0.92443
7200210 (epoch 1) shakespeare.txt │ \But thou wilt be avenged on m │ Tut theu ailt be a en ed tf ty │ loss: 1.19680
7200240 (epoch 1) shakespeare.txt │ ophet! My dear cousin,\I, like │ ree

And bade some bare'd matter. The pass ofther. Thou art nature
Have such things of it to die
That than when this is to a curnes me.

LEENATO:
And his arcious an the grace:
Osher bading in the lord,
We are saiden brames and wits the spired
The pence be this an acts makest.

SIMPEY:
His priece to dead your grown most wornd of three
It spares servast: and dht whe sace of heavent fauet. Sontongus thy mischeer,
If thou antedst but leave thend, fares.

PROTEUS:
Thalk you are a man it is the day of drank,
Thrse indo the common heart which is not ban. To said your leeps
And so many his tries: his by theself
He shall be sence with four father;
I have seephed hald of on your poor,
And helds it in the roughts to be
mearing and subelies, worsed! that,
his goind to be such fliants, and begone
our grace.
Is not find, Iramula in a wwrat hither,
And she what it half a bonours fils, huston underself,
That arms with lovers what, here, sprail,
To my be a sward, in my breather'd hise aw your sume
└────────

KeyboardInterrupt: 

<h1>Generate text</h1>

In [22]:
txt.print_text_generation_header()
ry = np.array([[txt.convert_from_alphabet(ord("K"))]])
rh = np.zeros([1, INTERNALSIZE * NLAYERS])
for k in range(1000):
    ryo, rh = sess.run([Yo, H], feed_dict={X: ry, pkeep: 1.0, Hin: rh, batchsize: 1})
    rc = txt.sample_from_probabilities(ryo, topn=10 if epoch <= 1 else 2)
    print(chr(txt.convert_to_alphabet(rc)), end="")
    ry = np.array([[rc]])
txt.print_text_generation_footer()


┌───────────────────────────────────Generating random text from learned state───────────────────────────────────┐
I:
Broke, sir, and to seef
Some scarely place it in hast;
And have I may that do sond but to to have desired
Old this her presant death.

LUCIUS:
Yea see, sell! Trousan and by his lase
As the tell me to me.-
HAMLET:
Why, thou this' wind with me? I must not flesh ye, betoild
I would mirtle hell to steek. O good wit, help,
How night matl second the poul hansed

MOTH:
And wis mast all the murterer. Therefore a should be cast
And we dad with my hand.

KANW OF:

LYNONIUS:
And your assed with their present should be done.
This will come him whas threat a deed for were soldeses.
It sast we are so all you.

Cornter:
Yencand, to speak what a gits fear to cest?
Will you was this: my goad well as hot as yours,
Murd my doungs to the love of me sore
As hele concented.

CORTIANS:
Would shall be the word!---

CORWILK:
What is the cassion?
Now we are no datching of your cood.

PASSINA:
I 